In [20]:
#Importing libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import requests
import seaborn as sns
import pprint, time
from sklearn.model_selection import train_test_split
nltk.download('treebank')
from nltk.tokenize import sent_tokenize
import string
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from collections import Counter
from spellchecker import SpellChecker
import random
from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers
import base64
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

nltk.download('punkt')

nltk.download('universal_tagset')

%matplotlib inline

punct = string.punctuation

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\halapets\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\halapets\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\halapets\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [21]:
#loading spacy
import spacy
nlp = spacy.load("en_core_web_lg")
nlp.max_length = 16000000

In [22]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)
stopwords

['were',
 'moreover',
 'did',
 'one',
 'upon',
 'yours',
 'latterly',
 'ever',
 'me',
 'used',
 "'m",
 'could',
 'nothing',
 'they',
 'whenever',
 'show',
 '‘d',
 'here',
 'in',
 'take',
 'next',
 'quite',
 'seem',
 'herself',
 'whereas',
 'over',
 'anyhow',
 'them',
 'always',
 'if',
 'put',
 'are',
 'was',
 'everything',
 'becomes',
 'twelve',
 'also',
 'up',
 'name',
 'formerly',
 'her',
 'three',
 '’re',
 'mostly',
 'myself',
 'nevertheless',
 "'re",
 'as',
 'whence',
 '‘re',
 'than',
 'noone',
 'often',
 'first',
 'through',
 'enough',
 'yourself',
 'nobody',
 'although',
 '’s',
 'is',
 'while',
 'whoever',
 'all',
 'to',
 'had',
 'my',
 'beyond',
 'such',
 'herein',
 'together',
 'who',
 'see',
 'after',
 'anyone',
 'move',
 'meanwhile',
 'now',
 'just',
 'whereafter',
 'there',
 'thus',
 'whereby',
 'almost',
 'back',
 'done',
 'when',
 'really',
 'no',
 'anywhere',
 'between',
 'four',
 'until',
 'eight',
 'on',
 'rather',
 'namely',
 'hereupon',
 'hereby',
 'yet',
 'mine',
 's

In [23]:
LatestMobileReviews1 = pd.read_csv('C:\\Users\\halapets\\Documents\\LJMU Research_Sentiment Analysis\\LatestMobileReviews.csv',encoding='latin1')
LatestMobileReviews1.head()

,Product,ReviewUpdated,Rating,Date,Website,TitleGoodBad,ProductPrice,ProductPriceRange,Year,positive_negative,...,Positive_Negative_Tag,LengthofReviews,SentimentPolarity,SentimentSubjectivity,SentimentPolarityClass,AvgSentPolarity,AvgSentimentPolarityClass,AvgSentimentSubjectivity,ReviewUpdated1,ReviewUpdated2
0,Xiaomi Redmi Note 3,"['yet', 'another', 'value', 'money', 'product'...",3.0,"Jul 17, 2016 10:59 PM",MouthShut,Yet another Value for Money product from Xiaomi,9999,Budget Range,2016.0,1,...,Positive Rating,344,-0.028571,0.576190,Negative Rating,0.403533,Positive Rating,NaN,Yet another Value for Money product from Xiaom...,Yet another Value for Money product from Xiaom...
1,Xiaomi Redmi Note 3,"['awesome', 'marvelous', 'mobile', 'go', 'valu...",5.0,"Jul 17, 2016 10:42 PM",MouthShut,Awesome,9999,Budget Range,2016.0,1,...,Positive Rating,276,0.780000,0.640000,Positive Rating,0.403533,Positive Rating,NaN,"Awesome Marvelous Mobile just go for it., Val...","Awesome Marvelous Mobile just go for it., Val..."
2,Xiaomi Redmi Note 3,"['power', 'pack', 'review', 'redmi', 'note', '...",5.0,"Jul 17, 2016 06:18 PM",MouthShut,Power pack review on Redmi Note 3,9999,Budget Range,2016.0,1,...,Positive Rating,322,0.474444,0.477037,Positive Rating,0.403533,Positive Rating,NaN,Power pack review on Redmi Note 3 REDMI NOTE ...,Power pack review on Redmi Note 3 REDMI NOTE ...
3,Xiaomi Redmi Note 3,"['flagship', 'smartfone', 'phone', 'looks', 'v...",4.0,"Jul 17, 2016 04:44 PM",MouthShut,Flagship smartfone under 13000rs,9999,Budget Range,2016.0,1,...,Positive Rating,333,0.502500,0.832500,Positive Rating,0.403533,Positive Rating,NaN,Flagship smartfone under 13000rs This phone l...,Flagship smartfone under 13000rs This phone l...
4,Xiaomi Redmi Note 3,"['boon', 'mobile', 'master', 'piece', 'economi...",5.0,"Jul 17, 2016 12:32 PM",MouthShut,Boon for mobile,9999,Budget Range,2016.0,1,...,Positive Rating,316,0.448571,0.627857,Positive Rating,0.403533,Positive Rating,NaN,Boon for mobile Master piece in such a econom...,Boon for mobile Master piece in such a econom...


In [24]:
LatestMobileReviews1.shape

(160057, 23)

In [25]:
#tokenization and text preprocessing
def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-" or token.pos_ != "AUX" or token.pos_ != "NUM" or token.pos_ != "SYM" or token.pos_ != "ADP"  :
            temp = token.lemma_.lower().strip()
        tokens.append(temp)
        
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct and token.isalpha():
            cleaned_tokens.append(token)
    return cleaned_tokens

In [26]:
text_data_cleaning("    Hello how are you. Like this video! Please advise $")

['hello', 'like', 'video', 'advise']

### TF-IDF Vectorization

In [27]:
#word vectorization using tfidf
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)

In [28]:
X = LatestMobileReviews1['ReviewUpdated1']
y= LatestMobileReviews1 ['positive_negative']
len(X)
len(y)

160057

In [29]:
X_transformed = tfidf.fit_transform(X)

In [30]:
X_transformed.shape

(160057, 101644)

In [31]:
y.shape

(160057,)

In [32]:
#Converting TFIDF Matrix to dataframe
#df_TFIDF = pd.DataFrame(X_transformed.toarray(), columns=tfidf.get_feature_names())

In [33]:
#df_TFIDF.shape

In [34]:
#writing dataframe 
#df_TFIDF.to_csv("df_TFIDF.csv",index =False, header = True)

### Ensemble Model on TFIDF Vectorization (Classifiers used- Decision Tree, Naive Bayes, Decision Tree and Logistic Regression)

In [35]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

In [36]:
seed = 7
kfold = model_selection.KFold(n_splits=3, random_state=seed)
# create the sub models
estimators = []
model1 = LogisticRegression()
estimators.append(('logistic', model1))
model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))
model3 = SVC()
estimators.append(('svm', model3))
model4 = BernoulliNB()
estimators.append(('NB', model4))
# create the ensemble model
ensemble = VotingClassifier(estimators)


C:\Users\halapets\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [37]:
results = model_selection.cross_val_score(ensemble, X_transformed, y, cv=kfold)
print(results.mean())#mean estimate of classification accuracy 0.9034468286080192

C:\Users\halapets\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\halapets\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit

0.9034468286080192


In [38]:
results.std()# Standard Deviation for classification accuracy 0.009175947317369745

0.009175947317369745

In [39]:
###------------------------------------------------------####--------------------------------------------------------------------

### Bag of Words

In [40]:
#word vectorization using tfidf
BOW = CountVectorizer(tokenizer = text_data_cleaning)

In [41]:
X = LatestMobileReviews1['ReviewUpdated']
y= LatestMobileReviews1 ['positive_negative']
len(X)

160057

In [42]:
X_transformed_BOW = BOW.fit_transform(X)

In [43]:
X_transformed_BOW.shape

(160057, 156147)

In [44]:
y.shape

(160057,)

In [45]:
#Converting TFIDF Matrix to dataframe
#df_BOW = pd.DataFrame(X_transformed_BOW.toarray(), columns=BOW.get_feature_names())

In [46]:
#df_BOW.shape

### Ensemble Model on Count Vectorization-Bag of Words (Classifiers used- Decision Tree, Naive Bayes, Decision Tree and Logistic Regression)

In [47]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier

In [48]:
seed = 7
kfold = model_selection.KFold(n_splits=3, random_state=seed)
# create the sub models
estimators = []
model1 = LogisticRegression()
estimators.append(('logistic', model1))
model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))
model3 = SVC()
estimators.append(('svm', model3))
model4 = BernoulliNB()
estimators.append(('NB', model4))
# create the ensemble model
ensemble = VotingClassifier(estimators)


C:\Users\halapets\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [49]:
results_BOW = model_selection.cross_val_score(ensemble, X_transformed_BOW, y, cv=kfold)
print(results_BOW.mean()) #mean estimate of classification accuracy 0.9035717859272561

C:\Users\halapets\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\halapets\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit

0.9035717859272561


In [50]:
results_BOW.std()# Standard Deviation for classification accuracy 0.008914551654112519

0.008914551654112519